# Task 8.4 Qiskit Runtime Rest API

## Objective 1 : Interact with the Qiskit IBM Runtime REST API

The Qiskit Runtime REST API provides programmatic access to IBM Quantum systems, allowing you to:

* **Run circuits on quantum processors (QPUs)**
* **Get information** about quantum processors, service instances, workloads and 
* **Manage jobs and sessions** for efficient quantum computing workflows
* **Access advanced features** like error mitigation and custom options

### Authentication

In [ ]:
import requests
import json
 
url = 'https://iam.cloud.ibm.com/identity/token'
api_key="<YOUR_API_KEY>"
data = f'grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={api_key}'
response = requests.post(url, data=data)

# Bearer token to authorize requests to the REST API
if response.status_code == 200:
        print(f"Access token obtained successfully.")
        bearer_token = response.json()['access_token']
else:
    print(f"Error obtaining bearer token:{response.status_code} ,{response.text}")
    bearer_token = None


### backends

In [ ]:
import requests
 
reqUrl = "https://quantum.cloud.ibm.com/api/v1/backends"

service_CRN = "<crn:YOUR_INSTANCE_CRN>"
 
headersList = {
  "Accept": "application/json",
  "Authorization": f"Bearer {bearer_token}",
  "Service-CRN": f"{service_CRN}",
  "IBM-API-Version":"2025-05-01"
}

payload = ""
 
response = requests.request("GET", reqUrl, data=payload,  headers=headersList)
 
if response.status_code == 200:
        print(f"Available Backends are:")
        print(response.json())
        backend = response.json()['devices'][0]
else:
    print(f"Error obtaining backends:{response.status_code} ,{response.text}")
    backend = None



### Submit a Job

In [ ]:
import requests
import json
 
reqUrl = "https://quantum.cloud.ibm.com/api/v1/jobs"
 
headersList = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": f"Bearer {bearer_token}",
  "Service-CRN": f"{service_CRN}",
  "IBM-API-Version":"2025-05-01"
}
 
circuit_qasm = '''OPENQASM 3.0; include "stdgates.inc"; bit[1] c; x $0; c[0] = measure $0;'''
 
payload = json.dumps({
  "program_id": "sampler",
  "backend": f"{backend}",
  "params": {
    "pubs": [[
      circuit_qasm
    ]],
    "version": 2
  }
})
 
response = requests.request("POST", reqUrl, data=payload,  headers=headersList)

if response.status_code == 200:
        print(f"Submitted Job:")
        print(response.json())
        job_id = response.json()['id']
else:
    print(f"Error submitting job:{response.status_code} ,{response.text}")
    job_id = None



### Retrieve Results

In [ ]:
import requests
import json
 
reqUrl = f"https://quantum.cloud.ibm.com/api/v1/jobs/{job_id}"
#reqUrl = f"https://quantum.cloud.ibm.com/api/v1/jobs/d337gou6pnbs739iac80/results"
 
headersList = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": f"Bearer {bearer_token}",
  "Service-CRN": f"{service_CRN}",
  "IBM-API-Version":"2025-05-01"
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

if response.status_code == 200:
        print(response.json())
        
else:
    print(f"Error retrieving job:{response.status_code} ,{response.text}")



### Use a Session

get session_id and use it in next job